# **ImageNet Dataset Images Classification**

The ImageNet dataset consists of 14 millions decent resolution (224 x 244 pixels each) images. The processing of this dataset requires a great amount of computing power in terms of CPU, GPU, and RAM. So, it is not suitable for everyday laptop. To provide more practice oppotunities for researchers and students, we will be using the Fast.ai's Imagenette Dataset.

# Fast.ai's Imagenette Dataset

Imagenette is a dataset that's extracted from the large ImageNet collection of images.

In [2]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

--2023-12-22 23:20:54--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.173.32, 52.216.217.144, 52.217.199.128, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.173.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1557161267 (1.5G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  35.0MB/s    in 42s     

2023-12-22 23:21:36 (35.6 MB/s) - ‘imagenette2.tgz’ saved [1557161267/1557161267]



In [3]:
!tar -xf imagenette2.tgz

**Load Images From Imagenette**

Use Keras ImageDataGenerator to load the large images from the Imagenette dataset without maxing out the RAM.

In [4]:
from keras.preprocessing.image import ImageDataGenerator

imagegen = ImageDataGenerator()
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


# VGG16 CNN Model Using Keras

VGG16 is a CNN architecture containing 16 layers in total, with 13 convolutional layers.

**Download VGG16 Weights**

In [5]:
from keras.applications import VGG16

# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

**Generate Features From VGG16**

In [6]:
from keras.utils import to_categorical
# extract train and val features
vgg_features_train = pretrained_model.predict(train)
vgg_features_val = pretrained_model.predict(val)

31/31 [==============================] - 2458s 79s/step


In [12]:
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

**Build The VGG16 CNN Model**

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

model = Sequential()
model.add(Flatten(input_shape=(7,7,512)))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

**Compile The Model**

In [14]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 100)               2508900   
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 100)               400       
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 2510310 (9.58 MB)
Trainable params: 2510110 (9.58 MB)
Non-trainable params: 200 (800.00 Byte)
____________

**Train The Model**

In [16]:
model.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))

Epoch 1/50
74/74 [==============================] - 9s 111ms/step - loss: 0.4407 - accuracy: 0.8830 - val_loss: 0.2012 - val_accuracy: 0.9383
Epoch 2/50
74/74 [==============================] - 6s 77ms/step - loss: 0.1307 - accuracy: 0.9763 - val_loss: 0.1749 - val_accuracy: 0.9478
Epoch 3/50
74/74 [==============================] - 7s 98ms/step - loss: 0.0595 - accuracy: 0.9936 - val_loss: 0.1724 - val_accuracy: 0.9485
Epoch 4/50
74/74 [==============================] - 6s 77ms/step - loss: 0.0343 - accuracy: 0.9974 - val_loss: 0.1674 - val_accuracy: 0.9470
Epoch 5/50
74/74 [==============================] - 7s 90ms/step - loss: 0.0224 - accuracy: 0.9985 - val_loss: 0.1713 - val_accuracy: 0.9434
Epoch 6/50
74/74 [==============================] - 6s 85ms/step - loss: 0.0154 - accuracy: 0.9990 - val_loss: 0.1681 - val_accuracy: 0.9473
Epoch 7/50
74/74 [==============================] - 10s 142ms/step - loss: 0.0130 - accuracy: 0.9996 - val_loss: 0.1734 - val_accuracy: 0.9475
Epoch 8/50

The accurancy after 50 epochs is 99.93%, making the VGG16 architecture of this CNN model suitable for the ImageNet dataset.